In [322]:
import warnings
warnings.filterwarnings(action='ignore')
import MySQLdb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [323]:
import requests
from bs4 import BeautifulSoup

In [324]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [325]:
# pip install html_table_parser

In [326]:
import requests
from bs4 import BeautifulSoup
from html_table_parser import parser_functions
import os

In [327]:
# # 날짜 dataframe 만들기
# start_date = pd.to_datetime('2015-01-01') ## 시작 날짜
# end_date = pd.to_datetime('2022-03-07') ## 마지막 날짜
 
# dates = pd.date_range(start_date,end_date,freq='M') ## 월단위로 생성
# dates = pd.DataFrame(dates, columns=['date'])

In [328]:
# 년,월,일 -> 날짜
def change_date(x):
    ret = []
    for da in x.split():
        ret.append(da[:-1])
    return '.'.join(ret)

In [329]:
# K, M, B 변환
def m_to_k(x):
    if x[-1] == 'M':
        return float(x[:-1]) * 1000
    elif x[-1] == 'K':
        return float(x[:-1])
    elif x[-1] == 'B':
        return float(x[:-1]) * 1000000
    else:
        return np.nan

In [330]:
# 나스닥 종합지수, 상해 종합지수, 달러지수 병합

In [331]:
## 나스닥
df_nasdaq = pd.read_csv('나스닥종합지수.csv')
df_nasdaq.drop(['오픈','고가','저가'], axis=1, inplace=True)
df_nasdaq.rename(columns={'날짜':'date', '종가':'nasdaq', '거래량':'nasdaq_거래량', '변동 %':'nasdaq_volatility'},
                 inplace=True)

### 날짜 전처리
df_nasdaq.date = df_nasdaq.date.apply(change_date)
df_nasdaq.date = pd.to_datetime(df_nasdaq.date)

### 가격 float타입
df_nasdaq.nasdaq = df_nasdaq.nasdaq.str.replace(',','').astype('float')

### 거래량 전처리
df_nasdaq['nasdaq_거래량'] = df_nasdaq['nasdaq_거래량'].apply(m_to_k)

### 변동% 전처리
df_nasdaq['nasdaq_volatility'] = df_nasdaq.nasdaq_volatility.apply(lambda x : float(x[:-1]))

In [332]:
## 상해 종합지수
df_ssec = pd.read_csv('상하이종합 내역.csv')
df_ssec.drop(['오픈','고가','저가'], axis=1, inplace=True)
df_ssec.rename(columns={'날짜':'date', '종가':'ssec', '거래량':'ssec_거래량', '변동 %':'ssec_volatility'},
               inplace=True)

### 날짜 전처리
df_ssec.date = df_ssec.date.apply(change_date)
df_ssec.date = pd.to_datetime(df_ssec.date)

### 가격 float타입
df_ssec.ssec = df_ssec.ssec.str.replace(',','').astype('float')

### 거래량 전처리
df_ssec['ssec_거래량'] = df_ssec['ssec_거래량'].apply(m_to_k)

### 변동% 전처리
df_ssec['ssec_volatility'] = df_ssec.ssec_volatility.apply(lambda x : float(x[:-1]))

In [333]:
## 달러지수
df_dollar = pd.read_csv('달러 지수.csv')
df_dollar.drop(['오픈','고가','저가', '거래량'], axis=1, inplace=True)
df_dollar.rename(columns={'날짜':'date', '종가':'dollar', '변동 %':'dollar_volatility'}, inplace=True)

### 날짜 전처리
df_dollar.date = df_dollar.date.apply(change_date)
df_dollar.date = pd.to_datetime(df_dollar.date)

### 변동% 전처리
df_dollar['dollar_volatility'] = df_dollar.dollar_volatility.apply(lambda x : float(x[:-1]))

In [334]:
## 병합
df_composite_index = pd.merge(df_nasdaq, df_ssec, how='outer', on='date')
df_composite_index = pd.merge(df_composite_index, df_dollar, how='outer', on='date')
# df_composite_index.to_csv('종합지수_merge.csv')

In [335]:
# 중국 2차전지, 전기차

In [336]:
## EVE Energy
df_eve = pd.read_csv('EVE Energy 주가.csv')
df_eve.drop(['오픈', '고가', '저가'], axis = 1, inplace = True)
df_eve.columns = 'date', 'EVE종가', 'EVE거래량', 'EVE변동률'
df_eve['date'] = pd.to_datetime(df_eve['date'].apply(change_date))
df_eve['EVE거래량'] = df_eve['EVE거래량'].apply(m_to_k).astype('float')
df_eve['EVE변동률'] = df_eve['EVE변동률'].apply(lambda x : x[:-1]).astype('float')

In [337]:
## BYD 주가
df_byd = pd.read_csv('BYD 주가.csv')
df_byd.drop(['오픈', '고가', '저가'], axis = 1, inplace=True)
df_byd.columns = 'date', 'BYD종가', 'BYD거래량', 'BYD변동률'
df_byd['BYD거래량'] = df_byd['BYD거래량'].apply(m_to_k).astype('float')
df_byd['date'] = pd.to_datetime(df_byd['date'].apply(change_date))
df_byd['BYD변동률'] = df_byd['BYD변동률'].apply(lambda x : x[:-1]).astype('float')

In [338]:
## eve, byd merge
merge_eve_byd = pd.merge(df_eve, df_byd, on = 'date', how = 'outer')
# merge_eve_byd.to_csv('중국2차전지.csv', header = False, index = False)

In [339]:
# 희토류, 리튬, 코발트 병합

In [340]:
## 리튬
df_li = pd.read_excel('리튬+가격2022-03-08.xls')
df_li.drop(['등락가', '등락비율', '재고량'], axis=1, inplace=True)
df_li.columns = ['date', 'li_price']
df_li.date = pd.to_datetime(df_li.date)
# df_li.to_csv('data_li.csv')

df_li = pd.read_csv('data_li.csv')
df_li.date = pd.to_datetime(df_li.date)

In [341]:
## 코발트
df_co = pd.read_csv('코발트 가격.csv')
df_co.drop(['오픈','고가','저가','거래량'], axis=1, inplace=True)
df_co.rename(columns={'날짜':'date', '종가':'co', '변동 %':'co_volatility'},
                 inplace=True)

### 날짜 전처리
df_co.date = df_co.date.apply(change_date)
df_co.date = pd.to_datetime(df_co.date)

### 가격 float타입
df_co.co = df_co.co.str.replace(',','').astype('float')

### 변동% 전처리
df_co['co_volatility'] = df_co.co_volatility.apply(lambda x : float(x[:-1]))

In [342]:
## 희토류
df_ra = pd.read_excel('희토류 가격.xlsx')
df_ra.drop(['등락가', '재고량', '등락비율'], axis = 1, inplace = True)
df_ra.columns = 'date', '희토류가격'
df_ra.date = pd.to_datetime(df_ra.date)

In [343]:
## 병합
merge_lico = pd.merge(df_li, df_co, on = 'date', how = 'outer')
merge_licora = pd.merge(merge_lico, df_ra, on = 'date', how = 'outer')
# merge_licora.to_csv('배터리원자재_merge.csv')

In [344]:
# 재무제표

In [345]:
## 재무제표1 크롤링
url = 'https://www.choicestock.co.kr/search/invest/TSLA/MRQ'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')
datas = soup.select('table.tableRanking')

## 데이터 프레임 생성
li = []
for data in datas:
    table = parser_functions.make2d(data)
    df = pd.DataFrame(data=table[1:], columns=table[0])
    li.append(df)
    
## 합병
df = pd.concat(li)

## 저장
# df.to_csv('fn_statement.csv')

In [346]:
## 재무제표1 전처리
df_fn_statements = pd.read_csv('tesla 재무제표.csv')
df_fn_statements = df_fn_statements.set_index('Unnamed: 1')
df_fn_statements = df_fn_statements.drop(['Unnamed: 0','초이스스탁US 프리미엄', '프리미엄', '무료'], axis = 1)
df_fn_statements.dropna(inplace = True)
df_fn_statements = df_fn_statements.T
df_fn_statements = df_fn_statements.astype('object')
df_fn_statements = df_fn_statements.reset_index()
df_fn_statements = df_fn_statements.rename_axis(columns={'Unnamed: 1' : ''})
df_fn_statements = df_fn_statements.rename(columns={'index':'date'})
df_fn_statements.date = df_fn_statements.date.apply(lambda x : x.replace('.','/'))
df_fn_statements.date = df_fn_statements.date.apply(lambda x : '/'.join(x.split('/')[::-1]))
df_fn_statements.date = pd.to_datetime(df_fn_statements.date)
df_fn_statements.drop(['중단손익'], axis=1, inplace=True)

drop_col = ['매입채무등', '단기차입금', '장기차입금', '이연수익', '이연세부채','기타부채', '부채총계',
            '이익잉여금', '기타포괄익', '자본총계', '영업활동', '투자활동', '재무활동', '환율변동',
            '현금의증감', '자본적지출', '잉여현금', '현금성자산', '단기투자', '매출채권등', '재고자산',
            '장기투자', '유형자산', '무형자산', '이연세자산', '기타자산','EBITDA', '법인세', '판매관리비',
            '연구개발비']

df_fn_statements.drop(drop_col, axis=1, inplace=True)
df_fn_statements = df_fn_statements.iloc[:28]

# float 타입변환
col = df_fn_statements.columns.values
for c in col[1:]:
    df_fn_statements.loc[:,c] = df_fn_statements.loc[:,c].apply(lambda x : x.replace(',',''))
    df_fn_statements[c] = df_fn_statements[c].astype('float')

# 이익률 계산
df_fn_statements['매출총이익률'] = (df_fn_statements['매출총이익']/df_fn_statements['매출액'])*100
df_fn_statements['영업이익률'] = (df_fn_statements['영업이익']/df_fn_statements['매출액'])*100
# df_fn_statements.to_csv('fn_statements.csv')

In [347]:
## 재무제표2 크롤링
url = 'https://www.choicestock.co.kr/search/invest/TSLA'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')
datas = soup.select('table')
li = []
for data in datas:
    table = parser_functions.make2d(data)
    df = pd.DataFrame(data=table[1:], columns=table[0])
    li.append(df)
### 병합
df = pd.concat(li)

In [348]:
## 재무제표2 전처리

In [349]:
df = df.iloc[:12, :].drop(['초이스스탁US 프리미엄', '프리미엄', '무료'], axis=1).set_index('').T
df.to_csv('재무재표2_전처리.csv')

In [350]:
df_fn_statements2 = pd.read_csv('재무재표2_전처리.csv')
df_fn_statements2 = df_fn_statements2.rename(columns={'Unnamed: 0':'date'})
df_fn_statements2.date = df_fn_statements2.date.apply(lambda x : x.replace('.','/'))
df_fn_statements2.date = df_fn_statements2.date.apply(lambda x : '/'.join(x.split('/')[::-1]))
df_fn_statements2.date = pd.to_datetime(df_fn_statements2.date)
df_fn_statements2.drop(['부채비율(%)','이자보상배수(배)'], axis=1, inplace=True)

In [351]:
# 리튬관련 etf

In [352]:
## LIT etf
df_lit = pd.read_csv('리튬 etf LIT.csv')
df_lit.drop(['Open', 'High', 'Low', 'Close'], axis = 1, inplace=True)
df_lit.columns = 'date', 'LIT조정종가', 'LIT거래량'
df_lit['date'] = pd.to_datetime(df_lit['date'])

In [353]:
## SGL ETF
df_sgl = pd.read_csv('리튬전지 SGL etf.csv')
df_sgl.drop(['오픈', '고가', '저가', '거래량'], axis = 1, inplace = True)
df_sgl.columns = 'date', 'SGL종가', 'SGL변동률'
df_sgl['date'] = df_sgl['date'].apply(change_date)
df_sgl['date'] = pd.to_datetime(df_sgl['date'])
### 변동% 전처리
df_sgl['SGL변동률'] = df_sgl.SGL변동률.apply(lambda x : float(x[:-1]))

In [354]:
## 리튬 etf 병합
df_lithium_etf = pd.merge(df_lit, df_sgl, on = 'date', how = 'outer')
# df_lithium_etf.to_csv('lithium_etf_merge')

In [355]:
# 반도체관련 etf

In [356]:
## psi etf (미국)
df_psi = pd.read_csv("미국 반도체 etf PSI.csv")
df_psi = df_psi.rename(columns={'날짜':'date'})
df_psi = df_psi.rename(columns={'거래량':'PSI거래량','종가':'PSI종가'})
df_psi.drop(['오픈','고가','저가','변동 %'], axis=1, inplace=True)
df_psi.date = pd.to_datetime(df_psi.date.apply(change_date))
df_psi.PSI거래량 = df_psi.PSI거래량.apply(m_to_k).astype('float')

In [357]:
## soxx etf (미국)
df_soxx = pd.read_csv("미국 반도체 etf SOXX.csv")
df_soxx = df_soxx.rename(columns = {'날짜' : 'date','거래량' : 'SOXX거래량','종가' : 'SOXX종가'})
df_soxx.drop(['오픈','고가','저가','변동 %'], axis=1, inplace=True)
df_soxx.date = pd.to_datetime(df_soxx.date.apply(change_date))
df_soxx.SOXX거래량 = df_soxx.SOXX거래량.apply(m_to_k).astype("float")

In [358]:
## soxl etf (미국)
df_soxl = pd.read_csv("미국 반도체 etf SOXL.csv")
df_soxl = df_soxl.rename(columns = {'날짜' : 'date','거래량' : 'SOXL거래량','종가' : 'SOXL종가'})
df_soxl.drop(['오픈','고가','저가','변동 %'], axis=1, inplace=True)
df_soxl.date = pd.to_datetime(df_soxl.date.apply(change_date))
df_soxl.SOXL거래량 = df_soxl.SOXL거래량.apply(m_to_k).astype("float")

In [359]:
## 반도체 etf merge
merge_outer = pd.merge(df_psi, df_soxx,how='outer', on='date')
semi_etf_merge = pd.merge(merge_outer,df_soxl, how='outer', on='date')
# semi_etf_merge.to_csv('반도체 etf merge.csv')

In [360]:
# 전기차관련 etf

In [361]:
## driv etf (미국)
df_driv = pd.read_csv("미국 전기차 erf DRIV.csv")
df_driv = df_driv.rename(columns = {'날짜':'date', '종가':'DRIV종가','거래량':'DRIV거래량'})
df_driv.drop(['오픈','고가','저가','변동 %'], axis =1, inplace=True)
df_driv.date = pd.to_datetime(df_driv.date.apply(change_date))
df_driv.DRIV거래량 = df_driv.DRIV거래량.apply(m_to_k).astype('float')

In [362]:
## acdc etf (캐나다)
df_acdc = pd.read_csv("캐나다 전기차 etf ACDC.csv")
df_acdc = df_acdc.rename(columns = {'날짜':'date', '종가':'ACDC종가','거래량':'ACDC거래량'})
df_acdc.drop(['오픈','고가','저가','변동 %'], axis =1, inplace=True)
df_acdc.date = pd.to_datetime(df_acdc.date.apply(change_date))
df_acdc.ACDC거래량 = df_acdc.ACDC거래량.apply(m_to_k).astype('float')

In [363]:
## kars (미국 자율주행 etf)
df_kars = pd.read_csv("미국 자율주행전기차 etf KARS.csv")
df_kars.drop(['오픈','고가','저가'], axis=1, inplace=True)
df_kars.rename(columns={'날짜':'date', '종가':'kars', '거래량':'kars_거래량', '변동 %':'kars_volatility'},
                 inplace=True)

### 날짜 전처리
df_kars.date = df_kars.date.apply(change_date)
df_kars.date = pd.to_datetime(df_kars.date)

### 가격 float타입
df_kars.kars = df_kars.kars.astype('float')

### 거래량 전처리
df_kars['kars_거래량'] = df_kars['kars_거래량'].apply(m_to_k)

### 변동% 전처리
df_kars['kars_volatility'] = df_kars.kars_volatility.apply(lambda x : float(x[:-1]))

In [364]:
## BATT (미국 배터리 etf)
df_batt = pd.read_csv("미국 배터리 etf BATT.csv")
df_batt.drop(['오픈','고가','저가'], axis=1, inplace=True)
df_batt.rename(columns={'날짜':'date', '종가':'batt', '거래량':'batt_거래량', '변동 %':'batt_volatility'},
                 inplace=True)

### 날짜 전처리
df_batt.date = df_batt.date.apply(change_date)
df_batt.date = pd.to_datetime(df_batt.date)

### 가격 float타입
df_batt.batt = df_batt.batt.astype('float')

### 거래량 전처리
df_batt['batt_거래량'] = df_batt['batt_거래량'].apply(m_to_k)

### 변동% 전처리
df_batt['batt_volatility'] = df_batt.batt_volatility.apply(lambda x : float(x[:-1]))

In [365]:
## 전기차 etf 병합
ele_car_merge = pd.merge(df_driv, df_acdc, how='outer', on='date')
ele_car_merge = pd.merge(ele_car_merge, df_kars, how='outer', on='date')
ele_car_merge = pd.merge(ele_car_merge, df_batt, how='outer', on='date')
# ele_car_merge.to_csv('전기차 etf merge.csv')

In [366]:
## TBX (미국채금리 인버스 etf)

In [367]:
df_tbx = pd.read_csv("미국채금리 인버스 etf TBX.csv")
df_tbx.drop(['오픈','고가','저가'], axis=1, inplace=True)
df_tbx.rename(columns={'날짜':'date', '종가':'tbx', '거래량':'tbx_거래량', '변동 %':'tbx_volatility'},
                 inplace=True)

### 날짜 전처리
df_tbx.date = df_tbx.date.apply(change_date)
df_tbx.date = pd.to_datetime(df_tbx.date)

### 가격 float타입
df_tbx.tbx = df_tbx.tbx.astype('float')

### 거래량 전처리
df_tbx['tbx_거래량'] = df_tbx['tbx_거래량'].apply(m_to_k)

### 변동% 전처리
df_tbx['tbx_volatility'] = df_tbx.tbx_volatility.apply(lambda x : float(x[:-1]))

In [368]:
# 한국 2차전지 etf

In [369]:
## kodex
df_kodex = pd.read_csv('한국 2차전지 etf KODEX 2차전지산업.csv')
df_kodex.drop(['오픈','고가','저가'], axis=1, inplace=True)
df_kodex.rename(columns={'날짜':'date', '종가':'kodex', '거래량':'kodex_거래량', '변동 %':'kodex_volatility'},
                 inplace=True)

### 날짜 전처리
df_kodex.date = df_kodex.date.apply(change_date)
df_kodex.date = pd.to_datetime(df_kodex.date)

### 가격 float타입
df_kodex.kodex = df_kodex.kodex.str.replace(',','').astype('float')

### 거래량 전처리
df_kodex['kodex_거래량'] = df_kodex['kodex_거래량'].apply(m_to_k)

### 변동% 전처리
df_kodex['kodex_volatility'] = df_kodex.kodex_volatility.apply(lambda x : float(x[:-1]))

In [370]:
## tiger
df_tiger = pd.read_csv('타이거 2차전지 etf.csv')
df_tiger.drop(['오픈','고가','저가'], axis=1, inplace=True)
df_tiger.rename(columns={'날짜':'date', '종가':'tiger', '거래량':'tiger_거래량', '변동 %':'tiger_volatility'},
                 inplace=True)

### 날짜 전처리
df_tiger.date = df_tiger.date.apply(change_date)
df_tiger.date = pd.to_datetime(df_tiger.date)

### 가격 float타입
df_tiger.tiger = df_tiger.tiger.str.replace(',','').astype('float')

### 거래량 전처리
df_tiger['tiger_거래량'] = df_tiger['tiger_거래량'].apply(m_to_k)

### 변동% 전처리
df_tiger['tiger_volatility'] = df_tiger.tiger_volatility.apply(lambda x : float(x[:-1]))

In [371]:
btr_merge = pd.merge(df_kodex, df_tiger, how='outer', on='date')

In [372]:
# 테슬라 주가(target)

In [373]:
## 테슬라
df_tesla = pd.read_csv('테슬라 주가.csv')
df_tesla.drop(['오픈','고가','저가'], axis=1, inplace=True)
df_tesla.rename(columns={'날짜':'date', '종가':'tesla', '거래량':'tesla_거래량', '변동 %':'tesla_volatility'},
                 inplace=True)

### 가격 float타입
df_tesla.tesla = df_tesla.tesla.str.replace(',','').astype('float')

### 날짜 전처리
df_tesla.date = df_tesla.date.apply(change_date)
df_tesla.date = pd.to_datetime(df_tesla.date)

### 거래량 전처리
df_tesla['tesla_거래량'] = df_tesla['tesla_거래량'].apply(m_to_k)

### 변동% 전처리
df_tesla['tesla_volatility'] = df_tesla.tesla_volatility.apply(lambda x : float(x[:-1]))

In [374]:
# 유가 선물지수, 탄소배출권

In [375]:
## 유가 선물지수
df_oli_price = pd.read_csv('국제 유가 선물.csv')
df_oli_price.drop(['오픈','고가','저가', '거래량'], axis=1, inplace=True)
df_oli_price.rename(columns={'날짜':'date', '종가':'oli_price', '변동 %':'oli_price_volatility'}, inplace=True)

### 날짜 전처리
df_oli_price.date = df_oli_price.date.apply(change_date)
df_oli_price.date = pd.to_datetime(df_oli_price.date)

### 변동% 전처리
df_oli_price['oli_price_volatility'] = df_oli_price.oli_price_volatility.apply(lambda x : float(x[:-1]))

In [376]:
## 탄소배출권
df_carbon_credits = pd.read_csv('탄소배출권.csv')
df_carbon_credits.drop(['오픈','고가','저가', '거래량'], axis=1, inplace=True)
df_carbon_credits.rename(columns={'날짜':'date', '종가':'carbon_credits', '변동 %':'carbon_credits_volatility'}, inplace=True)

### 날짜 전처리
df_carbon_credits.date = df_carbon_credits.date.apply(change_date)
df_carbon_credits.date = pd.to_datetime(df_carbon_credits.date)

### 변동% 전처리
df_carbon_credits['carbon_credits_volatility'] = df_carbon_credits.carbon_credits_volatility.apply(lambda x : float(x[:-1]))

In [377]:
## 유가, 탄소배출권 병합
merge_oli_carbon = pd.merge(df_oli_price, df_carbon_credits, how='outer', on='date')
# merge_oli_carbon.to_csv('유가 탄소 merge.csv')

In [378]:
# 미국 전기세

In [379]:
## 미국 전기세 전처리
df_e_fee_usa = pd.read_csv('미국 전기세.csv', skiprows=4)
df_e_fee_usa = df_e_fee_usa.drop(['units','source key'], axis=1)
df_e_fee_usa = df_e_fee_usa.set_index('description').T
df_e_fee_usa = pd.DataFrame(df_e_fee_usa['United States : industrial'])
df_e_fee_usa = df_e_fee_usa.reset_index()
df_e_fee_usa = df_e_fee_usa.rename(columns={'United States : industrial' : 'elec_fee', 'index':'date'})
df_e_fee_usa = df_e_fee_usa.iloc[168:]
df_e_fee_usa.date = pd.to_datetime(df_e_fee_usa['date'])
df_e_fee_usa.elec_fee = df_e_fee_usa.elec_fee.astype('float')

In [380]:
# 알류미늄, 구리, 신재생

In [381]:
## REGI(신재생)
### REGI 데이터 전처리
df_regi = pd.read_csv('./REGI 역사적 데이터/REGI 역사적 데이터.csv')
df_regi.drop(['오픈', '고가', '저가'], axis=1, inplace=True)
df_regi.columns = ['date', 'REGI_price', 'REGI_vol', 'REGI_change']
df_regi.date = df_regi.date.apply(change_date)
df_regi.date = pd.to_datetime(df_regi.date)
df_regi.REGI_vol = df_regi.REGI_vol.apply(m_to_k)
df_regi['REGI_change'] = df_regi['REGI_change'].apply(lambda x: float(x.replace('%', '')))

In [382]:
## 알루미늄 전처리
df_alu = pd.read_csv('알루미늄 내역.csv')
df_alu.drop(['오픈', '고가', '저가'], axis=1, inplace=True)
df_alu.columns = ['date', 'alu_price', 'alu_vol', 'alu_change']
df_alu.date = df_alu.date.apply(change_date)
df_alu.date = pd.to_datetime(df_alu.date)
df_alu['alu_price'] = df_alu['alu_price'].apply(lambda x: x.replace(',', ''))
df_alu['alu_price'] = pd.to_numeric(df_alu['alu_price'])
df_alu['alu_vol'] = df_alu['alu_vol'].apply(m_to_k)
df_alu['alu_change'] = df_alu['alu_change'].apply(lambda x: float(x.replace('%', '')))

In [383]:
## 구리 데이터 전처리
df_cu = pd.read_csv('영국 구리 선물 내역.csv')
df_cu.drop(['오픈', '고가', '저가', '거래량'], axis=1, inplace=True)
df_cu.columns = ['date', 'cu_price', 'cu_change']
df_cu.date = df_cu.date.apply(change_date)
df_cu.date = pd.to_datetime(df_cu.date)
df_cu.cu_price = df_cu.cu_price.apply(lambda x: float(x.replace(',', '')))
df_cu['cu_change'] = df_cu['cu_change'].apply(lambda x: float(x.replace('%', '')))

In [384]:
## 알류미늄, 구리, 신재생 병합
al_cu_regi_merge = pd.merge(df_alu, df_cu, on='date', how='outer')
al_cu_regi_merge = pd.merge(al_cu_regi_merge, df_regi, on='date', how='outer')
# al_cu_regi_merge = pd.read_csv('alu_cu_REGI_df.csv')
# al_cu_regi_merge.drop('Unnamed: 0', axis=1, inplace=True)
al_cu_regi_merge.date = pd.to_datetime(al_cu_regi_merge.date)

In [385]:
## S&P 500
df_snp_500 = pd.read_csv('S&P 500 내역.csv')
df_snp_500.drop(['오픈','고가','저가', '거래량'], axis=1, inplace=True)
df_snp_500.rename(columns={'날짜':'date', '종가':'snp_500', '거래량':'snp_500_거래량', '변동 %':'snp_500_volatility'},
               inplace=True)

### 날짜 전처리
df_snp_500.date = df_snp_500.date.apply(change_date)
df_snp_500.date = pd.to_datetime(df_snp_500.date)

### 가격 float타입
df_snp_500.snp_500 = df_snp_500.snp_500.str.replace(',','').astype('float')

### 변동% 전처리
df_snp_500['snp_500_volatility'] = df_snp_500.snp_500_volatility.apply(lambda x : float(x[:-1]))

In [ ]:
## 검색량

In [413]:
datas = os.listdir('./multiTimeline')
li = []
for data in datas:
    df = pd.read_csv('./multiTimeline/{}'.format(data))
    li.append(df)
df_con = pd.concat(li)
df_con = df_con.reset_index()
df_con = df_con.rename(columns={'index' : 'date', '카테고리: 모든 카테고리' : '검색량'})
df_con.검색량 = df_con.검색량.apply(lambda x: x if x.isnumeric() else np.nan)
df_con.dropna(inplace=True)
df_con.date = pd.to_datetime(df_con.date)
df_con.to_csv('검색량.csv')

In [386]:
# 테슬라 주가 기준 merge

In [387]:
## 테슬라 - 미국전기세
df_total = pd.merge(df_tesla, df_e_fee_usa, how='outer', on='date')
df_total = df_total.sort_values('date')
### 전기세의 nan값은 ffill(한달 단위기 때문에)
df_total.elec_fee.fillna(method='ffill', inplace=True)
df_total.dropna(how='any',inplace=True)

In [388]:
## 테슬라 - 유가,탄소
df_total = pd.merge(df_total, merge_oli_carbon, how='left', on='date')

In [389]:
## 테슬라 - 중국 전기차 주가
df_total = pd.merge(df_total, merge_eve_byd, how='left', on='date')

In [390]:
# ## 테슬라 - 전기차 etf
# df_total = pd.merge(df_total, ele_car_merge, how='left', on='date')

In [391]:
## 테슬라 - 반도체 etf
df_total = pd.merge(df_total, semi_etf_merge, how='left', on='date')

In [392]:
## 테슬라 - 리튬 etf
df_total = pd.merge(df_total, df_lithium_etf, how='left', on='date')

In [393]:
## 테슬라 - 희토류, 리튬, 코발트
df_total = pd.merge(df_total, merge_licora, how='left', on='date')

### 리튬 nan값 선형보간
df_total['li_price'] = df_total['li_price'].interpolate()

### 희토류 nan값 선형보간
df_total.희토류가격 = df_total.희토류가격.interpolate()

### 코발트 nan값 선형보간
df_total.co = df_total.co.interpolate()
df_total['co_volatility'] = df_total['co_volatility'].interpolate()

In [394]:
## 테슬라 - 종합지수
df_total = pd.merge(df_total, df_composite_index, how='left', on='date')

In [395]:
## 테슬라 - 알루미늄, 구리, 신재생
df_total = pd.merge(df_total, al_cu_regi_merge, how='left', on='date')

In [396]:
## 테슬라 - S&P 500
df_total = pd.merge(df_total, df_snp_500, how='left', on='date')

In [397]:
## 테슬라 - 2차전지
df_total = pd.merge(df_total, btr_merge, how='left', on='date')

In [398]:
## 테슬라 - tbx
df_total = pd.merge(df_total, df_tbx, how='left', on='date')

In [415]:
## 테슬라 - 검색량
df_total = pd.merge(df_total, df_con, how='left', on='date')

In [399]:
# ## 2차전지, 전기차 etf는 2019년부터 데이터가 있기 때문에 nan값을 따로 채우지 않고 2019년 이전의 데이터를 드랍
# ### 나머지 선형보간
# col_total = df_total.columns.values
# for col in col_total[1:]:
#     df_total[col] = df_total[col].interpolate()

# ### 나머지 nan값 처리
# df_total.fillna(method='bfill', inplace=True)

In [400]:
# # 2015년부터 있는 데이터만
# df_total.to_csv('테슬라 총합.csv')

In [416]:
# # 2019년부터 있는 데이터도 포함
# df_total.to_csv('테슬라 총합 2019.csv')